In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import lxml

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

# there are three tables on the page; the one with the postal codes 
# is the first one. If I were putting this in a production environment,
# where I was re-loading the data regularly, I would add some error checking
# to make sure the page's structure hadn't changed.

raw_postal_codes = df[0]
raw_postal_codes.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
